# 检验 reinit 函数

In [17]:
import numpy as np
import os
from fealpy.decorator import cartesian
from fealpy.mesh import UniformMesh2d
from scipy import ndimage

# 非符号距离函数的水平集函数
@cartesian
def phi_nsd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = (x - 0.5)**2 + (y - 0.75)**2 - 0.0115

    return val

# 符号距离函数的水平集函数
@cartesian
def phi_sd(p):
    x = p[..., 0]
    y = p[..., 1]
    val = np.sqrt((x - 0.5)**2 + (y - 0.75)**2) - 0.15

    return val

nelx, nely = 32, 20
domain = [0, 1, 0, 1]
hx = (domain[1] - domain[0]) / nelx
hy = (domain[3] - domain[2]) / nely
mesh = UniformMesh2d(extent=(0, nelx+2, 0, nely+2), 
                    h=(hx, hy), origin=(domain[0], domain[2]))

bc = mesh.entity_barycenter('cell')
#print("bc:", bc.shape, "\n", bc.round(3))
phi_nsd_interpolate = phi_nsd(bc)
#print("phi_nsd_interpolate:", phi_nsd_interpolate.shape, "\n", phi_nsd_interpolate.round(3))

phi_sd_interpolate = phi_sd(bc)
#print("phi_sd_interpolate:", phi_sd_interpolate.shape, "\n", phi_sd_interpolate.round(3))

# 非符号距离函数对应的单元密度
strucFull_nsd = (phi_nsd_interpolate >= 0).astype(int).reshape(nelx+2, nely+2).T
strucFull_nsd[0, :] = 0
strucFull_nsd[-1, :] = 0
strucFull_nsd[:, 0] = 0
strucFull_nsd[:, -1] = 0
print("strucFull_nsd:", strucFull_nsd.shape, "\n", strucFull_nsd)
struc_nsd = strucFull_nsd[1:-1, 1:-1]
print("struc_nsd:", struc_nsd.shape, "\n", struc_nsd)

# 符号距离函数对应的单元密度
strucFull_sd = (phi_sd_interpolate >= 0).astype(int).reshape(nelx+2, nely+2).T
strucFull_sd[0, :] = 0
strucFull_sd[-1, :] = 0
strucFull_sd[:, 0] = 0
strucFull_sd[:, -1] = 0
print("strucFull_sd:", strucFull_sd.shape, "\n", strucFull_sd)
struc_sd = strucFull_sd[1:-1, 1:-1]
print("struc_sd:", struc_sd.shape, "\n", struc_sd)

error_sd_nsd = np.sum(np.abs(struc_nsd - struc_sd))
print("error_sd_nsd:", error_sd_nsd)

def reinit(struc):
    """
    根据给定的结构重置化水平集函数.

    该函数通过添加 void 单元的边界来扩展输入结构，计算到最近的 solid 和 void 单元
    的欧几里得距离，并计算水平集函数，该函数在 solid phase 内为正，在 void phase 中为负

    Parameters:
    - struc ( ndarray - (nely, nelx) ): 表示结构的 solid(1) 和 void(0) 单元

    Returns:
    - lsf ( ndarray - (nely+2, nelx+2) ): 表示重置化后的水平集函数
    """

    strucFull = np.zeros((nely + 2, nelx + 2))
    strucFull[1:-1, 1:-1] = struc

    # Compute the distance to the nearest void (0-valued) cells.
    dist_to_0 = ndimage.distance_transform_edt(strucFull)

    # Compute the distance to the nearest solid (1-valued) cells.
    dist_to_1 = ndimage.distance_transform_edt(strucFull - 1)

    # Offset the distances by 0.5 to center the level set function on the boundaries.
    element_length = nelx / (2*nelx)
    temp_0 = dist_to_0 - element_length
    temp_1 = dist_to_1 - element_length

    # Calculate the level set function, ensuring the correct sign inside and outside the structure.
    #lsf = -(~strucFull.astype(bool)).astype(int) * temp_1 + strucFull * temp_0
    lsf = -(1 - strucFull) * temp_1 + strucFull * temp_0

    return lsf

lsf = reinit(struc_nsd)
print("lsf:", lsf.shape, "\n", lsf.round(3))

strucFull_lsf = (lsf.flatten('F') >= 0).astype(int).reshape(nelx+2, nely+2).T
strucFull_lsf[0, :] = 0
strucFull_lsf[-1, :] = 0
strucFull_lsf[:, 0] = 0
strucFull_lsf[:, -1] = 0
print("strucFull_lsf:", strucFull_lsf.shape, "\n", strucFull_lsf)
struc_lsf = strucFull_lsf[1:-1, 1:-1]
print("struc_lsf:", struc_lsf.shape, "\n", struc_lsf)

error_sd_lsf = np.sum(np.abs(struc_sd - struc_lsf))
print("error_sd_lsf:", error_sd_lsf)

error_nsd_lsf = np.sum(np.abs(struc_nsd - struc_lsf))
print("error_nsd_lsf:", error_nsd_lsf)

visualization_dir = 'visualization/'
os.makedirs(visualization_dir, exist_ok=True)

fname = os.path.join(visualization_dir, 'test_chaills_reinit.vts')
mesh.to_vtk(filename=fname, 
                celldata={'strucFull_nsd': strucFull_nsd.flatten('F'),
                          'strucFull_sd': strucFull_sd.flatten('F'),
                          'phi_nsd': phi_nsd_interpolate,
                          'phi_sd': phi_sd_interpolate,
                          'strucFull_lsf': strucFull_lsf.flatten('F'),
                          'lsf': lsf.flatten('F')})

strucFull_nsd: (22, 34) 
 [[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 1 1

'visualization/test_chaills_reinit.vts'

In [2]:
import numpy as np
np.sqrt(0.0225)

0.15